In [1]:
import folium
import pandas as pd
import json
import os
import jenkspy

# Employee Dataframe

In [2]:
# Load the spreadsheet
file_path = 'allactive.xlsx'

# Load the SortedSummary sheet into a DataFrame
df_sorted_summary = pd.read_excel(file_path, sheet_name='Summary')

# Extract the ID and Count columns
employee_counts = df_sorted_summary[['ID', 'Count']]

# Function to prepend 'PH' or 'PH0' based on the length of the ID
def prepend_ph(id_value):
    id_str = str(id_value)
    if len(id_str) == 9:
        return 'PH' + id_str
    elif len(id_str) == 8:
        return 'PH0' + id_str
    else:
        return id_str

# Apply the function to the ID column
employee_counts.loc[:, 'ID'] = employee_counts['ID'].apply(prepend_ph)

# Display the first few rows of the modified DataFrame
print(employee_counts.head())

            ID  Count
0  PH137603004      4
1  PH137607021      1
2  PH137603001      8
3  PH137603006      4
4  PH137604007      3


In [3]:
# Initialization code
jsonfolder_path = ['/Users/jerritor2/Desktop/EMS_Data/Bus_Analysis/philippines-json-maps/2023/geojson/municities/hires',
                  '/Users/jerritor2/Desktop/EMS_Data/Bus_Analysis/philippines-json-maps/2019/geojson/barangays/hires']

m = folium.Map()

def generateMapLocation(mapfocusindex):
    global m
    
    if mapfocusindex == 0: # all ph
        m = folium.Map(location=[12.616870715056407, 122.34833708101706], zoom_start=5.5)
        print("Setting focus to all of Philippines")
    elif mapfocusindex == 1: # calabarzon + metro manila
        m = folium.Map(location=[14.468224838723954, 121.04553358573668], zoom_start=10)
        print("Setting focus to Calabarzon + Metro Manila")
    else: # calabarzon centered on calamba
        m = folium.Map(location=[14.274281440588128, 121.05943840821561], zoom_start=11)
        print("Setting focus to Calabarzon")

def generateMapLevel(fpathindex):
    if fpathindex == 0:
        print("Setting borders to 2023 Municipalities")
    else:
        print("Setting borders to 2019 Barangays")
    
    # Read and store GeoJSON data
    geojson_data = {'type': 'FeatureCollection', 'features': []}
    geojson_dict = {}
    for filename in os.listdir(jsonfolder_path[fpathindex]):
        if filename.endswith('.json'):
            file_path = os.path.join(jsonfolder_path[fpathindex], filename)
            with open(file_path, 'r') as file:
                data = json.load(file)
                for feature in data['features']:
                    pcode = feature['properties']['ADM4_PCODE']
                    geojson_dict[pcode] = feature
                    geojson_data['features'].append(feature)
    
    print("GeoJSON data loaded.")
    return geojson_dict

def generateMap(focusindex, pathindex):
    generateMapLocation(focusindex)
    geojson_dict = generateMapLevel(pathindex)
    return geojson_dict

# Load the GeoJSON data for barangays
geojson_dict = generateMap(2, 1)

# Ensure ID column is string
employee_counts.loc[:, 'ID'] = employee_counts['ID'].astype(str)

# Filter GeoJSON features to include only those with matching IDs in the employee_counts DataFrame
filtered_features = [geojson_dict[code] for code in employee_counts['ID'] if code in geojson_dict]

# Add the count to the properties of each feature, converting it to a regular Python int
for feature in filtered_features:
    pcode = feature['properties']['ADM4_PCODE']
    feature['properties']['Count'] = int(employee_counts.loc[employee_counts['ID'] == pcode, 'Count'].values[0])

#all_features = [geojson_dict[code] for code in employee_counts['ID']]

# Create a new GeoJSON structure with filtered features
filtered_geojson_data = {'type': 'FeatureCollection', 'features': filtered_features}

############
# Create a choropleth map using Folium
folium.Choropleth(
    geo_data=filtered_geojson_data,
    name='choropleth',
    data=employee_counts,
    columns=['ID', 'Count'],
    key_on='feature.properties.ADM4_PCODE',
    nan_fill_color='blue',
    #nan_fill_opacity=0.5,
    fill_color='OrRd',
    #fill_opacity=0.7,
    #line_opacity=0.2,
    highlight=True,
    use_jenks=True,
    #bins=[1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,25,27,36,41,55],
    legend_name='Employee Count'
).add_to(m)


# Add tooltips to the GeoJSON layer
folium.GeoJson(
    filtered_geojson_data,
    #all_features,
    name='barangay_overlays',
    style_function=lambda x: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 0.5,
        'opacity': 0.5
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['ADM1_EN','ADM2_EN','ADM3_EN','ADM4_EN', 'Count'],
        aliases=['Region: ','Province: ','Municipality: ','Barangay: ', 'Employee Count: '],
        fill_opacity=0,
        line_opacity=0,
        localize=True
    )
).add_to(m)

Setting focus to Calabarzon
Setting borders to 2019 Barangays
GeoJSON data loaded.


In [4]:
# Add layer control
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('allactive.html')

In [ ]:
# Display the map in Jupyter Notebook (if running in such an environment)
m